In [1]:
from PIL import Image
import torch
cudas = torch.cuda.device_count()
for i in range(cudas):
    print(torch.cuda.get_device_properties(i).name)
if cudas < 1: 
    print("No GPU found")
    exit(1)

try: print("Is cuda available? ", torch.cuda.is_available())
except Exception as e: print("Error checking cuda availability: ", e)

try: print("Is MPS? ", torch.backends.mps.is_available())
except Exception as e: print("Error checking mips availability: ", e)

try: print("Is using HIP version? ", torch.version.hip)
except Exception as e: print("Error checking hip availability: ", e)

try: print("Is using CUDA version? ", torch.version.cuda)
except Exception as e: print("Error checking cuda availability: ", e)

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.workspace_limit = 256 * 1024 * 1024

Radeon RX 7900 XT
AMD Radeon Graphics
Is cuda available?  True
Is MPS?  False
Is using HIP version?  6.2.41134-65d174c3e
Is using CUDA version?  None


In [ ]:
from TranslatedLoader import GetDataset

csv_dir = '~/KNN/data/data_list/data_list/'
translation_table = {
    './val/': '~/KNN/data/valRGB/',
    './train/indoors': '~/KNN/data/trainRGB/train/indoors',
    './train_normal/indoors': '~/KNN/data/trainNorm/indoors',
    './val_normal/': '~/KNN/data/valNorm/',
    './train/outdoor': '~/KNN/data/trainRGBHDD/train/outdoor',
    './train_normal/outdoor': '~/KNN/data/trainNormHDD/outdoor',
}


In [ ]:
from model_unet import UNet
from train_model import TrainModel, LoadModel, visualize_predictions, ValidateModel
from vgg_model import VGGNormal
from dataset import load_data
from WriteWrapper import WriteWrapper
from torch.utils.data import DataLoader
import os

FORCE_TRAIN = True
FORCE_FULL_TRAIN = False
full_model_name = 'model_MSELoss_L_True_G_True_f_0.2_final.pth'
file_out = WriteWrapper('vis_log.txt')

validatorIndoor = GetDataset(csv_dir + 'val_indoors.csv', translation_table, 128)
validatorOutdoor = GetDataset(csv_dir + 'val_outdoor.csv', translation_table, 128)
validatorBoth = GetDataset(csv_dir + 'val_indoors.csv', translation_table, 128, more=csv_dir + 'val_outdoor.csv')
model = LoadModel(UNet(file_out, useLarge=True, useGlobalFeatures=True), full_model_name, True, '')

image_ids = []
for i in range(len(validatorIndoor)): image_ids.append(i)
visualize_predictions(image_ids, validatorIndoor, model, len(validatorIndoor), store_dir='vis_indoor')
validator_loader = DataLoader(validatorIndoor, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)
ValidateModel(model, validator_loader)
print("Indoor validation done", len(validatorIndoor))
print('-' * 20)

image_ids = []
for i in range(len(validatorOutdoor)): image_ids.append(i)
visualize_predictions(image_ids, validatorOutdoor, model, len(validatorOutdoor), store_dir='vis_outdoor')
validator_loader = DataLoader(validatorOutdoor, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)
ValidateModel(model, validator_loader)
print("Outdoor validation done", len(validatorOutdoor))
print('-' * 20)

image_ids = []
for i in range(len(validatorBoth)): image_ids.append(i)
validator_loader = DataLoader(validatorBoth, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)
ValidateModel(model, validator_loader)
print("Both validation done", len(validatorBoth))

MIOpen(HIP): Warning [IsEnoughWorkspace] [GetSolutionsFallback WTI] Solver <GemmFwdRest>, workspace required: 13107200, provided ptr: 0 size: 0
MIOpen(HIP): Warning [IsEnoughWorkspace] [EvaluateInvokers] Solver <GemmFwdRest>, workspace required: 13107200, provided ptr: 0 size: 0
MIOpen(HIP): Warning [IsEnoughWorkspace] [GetSolutionsFallback WTI] Solver <GemmFwdRest>, workspace required: 26214400, provided ptr: 0 size: 0
MIOpen(HIP): Warning [IsEnoughWorkspace] [EvaluateInvokers] Solver <GemmFwdRest>, workspace required: 26214400, provided ptr: 0 size: 0
MIOpen(HIP): Warning [IsEnoughWorkspace] [GetSolutionsFallback WTI] Solver <GemmFwdRest>, workspace required: 52428800, provided ptr: 0 size: 0
MIOpen(HIP): Warning [IsEnoughWorkspace] [EvaluateInvokers] Solver <GemmFwdRest>, workspace required: 52428800, provided ptr: 0 size: 0
MIOpen(HIP): Warning [IsEnoughWorkspace] [GetSolutionsFallback WTI] Solver <GemmFwdRest>, workspace required: 104857600, provided ptr: 0 size: 0
MIOpen(HIP): Wa

Validation Step [10/325], Loss: 52.1042
Validation Step [60/325], Loss: 65.4298
Validation Step [110/325], Loss: 51.5795
Validation Step [160/325], Loss: 63.9015
Validation Step [210/325], Loss: 47.1195
Validation Step [260/325], Loss: 47.3114
Validation Step [310/325], Loss: 49.7698
Validation finished Average Loss: 42.6904
Indoor validation done 325
--------------------
Validation Step [10/446], Loss: 65.7523
Validation Step [60/446], Loss: 66.7277
Validation Step [110/446], Loss: 64.2181
Validation Step [160/446], Loss: 73.7788
Validation Step [210/446], Loss: 78.4102
Validation Step [260/446], Loss: 81.0266
Validation Step [310/446], Loss: 68.7754
Validation Step [360/446], Loss: 70.8841
Validation Step [410/446], Loss: 84.6410
Validation finished Average Loss: 70.2750
Outdoor validation done 446
--------------------
Validation Step [10/771], Loss: 52.1042
Validation Step [60/771], Loss: 65.4298
Validation Step [110/771], Loss: 51.5795
Validation Step [160/771], Loss: 63.9015
Valid